In [102]:
import re
import math

In [103]:
import requests
import json

HOST = "http://10.35.0.112:8000/"
API_CANCLED = "canclled"
API_PNR_RANKING = "pnr_ranking?flight_id="
API_ALT_FLIGHT="alt_flight?flight_id="

def get_cancled():
    response = requests.request("GET", HOST + API_CANCLED, headers={}, data={})
    return json.loads(response.text)

def get_pnrs(flight_id):
    response = requests.request("GET", HOST + API_PNR_RANKING + str(flight_id), headers={}, data={})
    return json.loads(response.text)

def get_alt_flights(flight_id):
    response = requests.request("GET", HOST + API_ALT_FLIGHT + str(flight_id), headers={}, data={})
    return json.loads(response.text)

In [104]:
cancel=get_cancled()
len(cancel)
x=get_alt_flights('61d83160')
x

{'error': 'Flight does not exist'}

In [105]:
map_cabin={ 'F':'F','P':'F','C':'B','J':'B','Z':'B','Q':'P','R':'P','S':'P','T':'P','H':'P','M':'P',
          'Y':'E', 'A':'E', 'B':'E', 'D':'E', 'E':'E', 'G':'E', 'I':'E', 'K':'E', 'L':'E', 'N':'E', 'O':'E',
           'U':'E', 'V':'E','W':'E','X':'E'}
cabins=['F','B','P','E']
cabin_ind={'F':0 , 'B':1,'P':2,'E':3}

class_ind = {'F':'0', 'P':'1', 'C':'2', 'J':'3', 'Z':'4', 'Q':'5', 'R':'6', 'S':'7', 'T':'8', 'H':'9', 'M':'10',
             'Y':'11', 'A':'12', 'B':'13', 'D':'14', 'E':'15', 'G':'16', 'I':'17', 'K':'18', 'L':'19', 'N':'20', 'O':'21',
             'U':'22', 'V':'23', 'W':'24', 'X':'25'}

ind_class = {'0': 'F', '1': 'P', '2': 'C', '3': 'J', '4': 'Z', '5': 'Q', '6': 'R', '7': 'S', '8': 'T', '9': 'H', '10': 'M',
             '11': 'Y', '12': 'A', '13': 'B', '14': 'D', '15': 'E', '16': 'G', '17': 'I', '18': 'K', '19': 'L', '20': 'N', '21': 'O',
            '22': 'U', '23': 'V', '24': 'W', '25': 'X'}

In [106]:
# def preprocess_data(get_pnrs,alt_flight )
pnr={}
alt_flight={}
# pnr_flight={}
for f in cancel['data']:
#     print(f['flight_id'])
    pnr[f['flight_id']]=get_pnrs(f['flight_id'])["data"]
    alt=get_alt_flights(f['flight_id'])
    f2=[]
    for flight in alt['data']:
        f3={}
        f3['flight_id']=flight['flight_id']
        f3['delay']=flight['delay']
        f3['flight_time']=flight['flight_time']
        f3['F']=0
        f3['B']=0
        f3['P']=0
        f3['E']=0
#         print()
        for key in map_cabin:
            if key in flight['avilable_seats']:
                f3[map_cabin[key]]+=flight['avilable_seats'][key]
        f2.append(f3)
    f4=[]
    for flight in alt['c_flights']:
        f5=[]
        for fl in flight:
            f3={}
            f3['flight_id']=fl['flight_id']
            f3['delay']=fl['delay']
            f3['flight_time']=fl['flight_time']
            f3['F']=0
            f3['B']=0
            f3['P']=0
            f3['E']=0
#         print()
            for key in map_cabin:
                if key in fl['avilable_seats']:
                    f3[map_cabin[key]]+=fl['avilable_seats'][key]
            f5.append(f3)
        f4.append(f5)
    f6=[]
    for flight in alt['t_flights']:
        f7=[]
        for fl in flight:
            f3={}
            f3['flight_id']=fl['flight_id']
            f3['delay']=fl['delay']
            f3['flight_time']=fl['flight_time']
            f3['F']=0
            f3['B']=0
            f3['P']=0
            f3['E']=0
#         print()
            for key in map_cabin:
                if key in fl['avilable_seats']:
                    f3[map_cabin[key]]+=fl['avilable_seats'][key]
            f7.append(f3)
        f6.append(f7)
#     print(f['flight_id'])
    alt_flight[f['flight_id']]={'n_flights':f2,'c_flights':f4,'t_flights':f6}

In [107]:
# for k in alt_flight:
#     print(k)

In [108]:
used_seat={}
max_seat={}

for i in alt_flight:
    for j in alt_flight[i]['n_flights']:
        used_seat[j['flight_id']]={ 'F':'0','P':'0','C':'0','J':'0','Z':'0','Q':'0','R':'0','S':'0','T':'0','H':'0','M':'0',
          'Y':'0', 'A':'0', 'B':'0', 'D':'0', 'E':'0', 'G':'0', 'I':'0', 'K':'0', 'L':'0', 'N':'0', 'O':'0',
           'U':'0', 'V':'0','W':'0','X':'0'}
    for j in alt_flight[i]['c_flights']:
        used_seat[j[0]['flight_id']]={ 'F':'0','P':'0','C':'0','J':'0','Z':'0','Q':'0','R':'0','S':'0','T':'0','H':'0','M':'0',
          'Y':'0', 'A':'0', 'B':'0', 'D':'0', 'E':'0', 'G':'0', 'I':'0', 'K':'0', 'L':'0', 'N':'0', 'O':'0',
           'U':'0', 'V':'0','W':'0','X':'0'}
        used_seat[j[1]['flight_id']]={ 'F':'0','P':'0','C':'0','J':'0','Z':'0','Q':'0','R':'0','S':'0','T':'0','H':'0','M':'0',
          'Y':'0', 'A':'0', 'B':'0', 'D':'0', 'E':'0', 'G':'0', 'I':'0', 'K':'0', 'L':'0', 'N':'0', 'O':'0',
           'U':'0', 'V':'0','W':'0','X':'0'}
    for j in alt_flight[i]['t_flights']:
        used_seat[j[0]['flight_id']]={ 'F':'0','P':'0','C':'0','J':'0','Z':'0','Q':'0','R':'0','S':'0','T':'0','H':'0','M':'0',
          'Y':'0', 'A':'0', 'B':'0', 'D':'0', 'E':'0', 'G':'0', 'I':'0', 'K':'0', 'L':'0', 'N':'0', 'O':'0',
           'U':'0', 'V':'0','W':'0','X':'0'}
        used_seat[j[1]['flight_id']]={ 'F':'0','P':'0','C':'0','J':'0','Z':'0','Q':'0','R':'0','S':'0','T':'0','H':'0','M':'0',
          'Y':'0', 'A':'0', 'B':'0', 'D':'0', 'E':'0', 'G':'0', 'I':'0', 'K':'0', 'L':'0', 'N':'0', 'O':'0',
           'U':'0', 'V':'0','W':'0','X':'0'}
        used_seat[j[2]['flight_id']]={ 'F':'0','P':'0','C':'0','J':'0','Z':'0','Q':'0','R':'0','S':'0','T':'0','H':'0','M':'0',
          'Y':'0', 'A':'0', 'B':'0', 'D':'0', 'E':'0', 'G':'0', 'I':'0', 'K':'0', 'L':'0', 'N':'0', 'O':'0',
           'U':'0', 'V':'0','W':'0','X':'0'}
        

In [109]:
used_seat={}
maximum_seat={}

for i in alt_flight:
    for j in alt_flight[i]['n_flights']:
        used_seat[j['flight_id']]={'F':0,'B':0,'P':0,'E':0}
        maximum_seat[j['flight_id']]={'F':j['F'],'B':j['B'],'P':j['P'],'E':j['E']}
    for j in alt_flight[i]['c_flights']:
        used_seat[j[0]['flight_id']]={'F':0,'B':0,'P':0,'E':0}
        maximum_seat[j[0]['flight_id']]={'F':j[0]['F'],'B':j[0]['B'],'P':j[0]['P'],'E':j[0]['E']}
        used_seat[j[1]['flight_id']]={'F':0,'B':0,'P':0,'E':0}
        maximum_seat[j[1]['flight_id']]={'F':j[1]['F'],'B':j[1]['B'],'P':j[1]['P'],'E':j[1]['E']}
    for j in alt_flight[i]['t_flights']:
        used_seat[j[0]['flight_id']]={'F':0,'B':0,'P':0,'E':0}
        maximum_seat[j[0]['flight_id']]={'F':j[0]['F'],'B':j[0]['B'],'P':j[0]['P'],'E':j[0]['E']}
        used_seat[j[1]['flight_id']]={'F':0,'B':0,'P':0,'E':0}
        maximum_seat[j[1]['flight_id']]={'F':j[1]['F'],'B':j[1]['B'],'P':j[1]['P'],'E':j[1]['E']}
        used_seat[j[2]['flight_id']]={'F':0,'B':0,'P':0,'E':0}
        maximum_seat[j[2]['flight_id']]={'F':j[2]['F'],'B':j[2]['B'],'P':j[2]['P'],'E':j[2]['E']}
        

In [110]:
# used_seat

In [111]:
needed_seat=[]
for i in pnr:
    alloc={}
    for j in pnr[i]:
        alloc=j
        alloc['flight_id']=i
        needed_seat.append(alloc)
sorted_pnr = sorted(needed_seat, key=lambda x: x['score'],reverse=True)

In [112]:
allocated={}

for i in sorted_pnr:
    allocated[i['pnr']]='NULL'

In [113]:
# alt_flight

In [118]:
def get_delay_cost(delay_str):
    print(delay_str)
    comp = delay_str.split(", ")
    hours=0
    minutes=0
    seconds=0
    total_hours_from_days=0
    # Extract hours, minutes, and seconds from the time string
    if len(comp)==1:
        hours, minutes, seconds = map(int, comp[0].split(":"))
        otal_hours_from_days=0
    else:
        days, time_str = delay_str.split(", ")
        hours, minutes, seconds = map(int, comp[1].split(":"))
        total_hours_from_days = int(comp[0].split()[0]) * 24
    
    # Calculate total hours
    time = total_hours_from_days + hours
    if(minutes>30):
        time+=1
    time = math.exp(time/3)
    return time

def get_layoff_cost(delay_str):
    print(delay_str)
    comp = delay_str.split(", ")
    
    # Extract hours, minutes, and seconds from the time string
    hours=0
    minutes=0
    seconds=0
    total_hours_from_days=0
    if len(comp)==1:
        hours, minutes, seconds = map(int, comp[0].split(":"))
        total_hours_from_days=0
    else:
        days, time_str = delay_str.split(", ")
        hours, minutes, seconds = map(int, comp[1].split(":"))
        total_hours_from_days = int(comp[0].split()[0]) * 24
    
    # Calculate total hours
    time = total_hours_from_days + hours
    if(minutes>30):
        time+=1
    time = math.exp(time/2.5)
    return time


def get_flight_time_score(d,s):
    delay_str = s
    print(delay_str)
    comp = delay_str.split(", ")
    hours=0
    minutes=0
    seconds=0
    total_hours_from_days=0
    # Extract hours, minutes, and seconds from the time string
    if len(comp)==1:
        hours, minutes, seconds = map(int, comp[0].split(":"))
        total_hours_from_days=0
    else:
        days, time_str = delay_str.split(", ")
        hours, minutes, seconds = map(int, comp[1].split(":"))
        total_hours_from_days = int(comp[0].split()[0]) * 24
    
    # Calculate total hours
    time = total_hours_from_days + hours
    if(minutes>30):
        time+=1
    if d==0 :
        return 0
    elif(d>0):
        return d*math.exp(time/1.2)
    else:
        return (-d)*math.exp(time/1.5)

    

In [119]:
tot_cost=0

In [120]:
tot_cost=0
def obj(flights,pnr):
    global tot_cost
    n_flights=flights['n_flights']
    c_flights=flights['c_flights']
    t_flights=flights['t_flights']
    
#     c_flights=[]
#     print(n_flights[0])
#     print(c_flights[0])
#     return
#     print(len(n_flights))
#     print(len(c_flights))
#     print(n_flights)
    f_id=[-1]
    c_id=[-1]
    val=1e18
    c_up=3
    c_down=2
    temp_id=0
    cabin0=cabin_ind[map_cabin[pnr['class']]]
    for f in  n_flights:
#         print(f)
#         break
        cost=get_delay_cost(f['delay'])
            
        for i in cabins:
            if(f[i]<pnr['pax']+used_seat[f['flight_id']][i]):
                continue
            d=cabin_ind[i]-cabin0
            c=get_flight_time_score(d,f['flight_time'])
#             print(c+cost)
            if(c+cost<val):
                val=c+cost
                f_id=[f['flight_id']]
                c_id=[i]
#             temp_id+=1
#     flight_no=temp_id/n_class
#     class_no=temp_id%n_class
#     print('kkkkkk')
    for f in c_flights:
        
#         print(f[1]['delay'])
        cost=get_delay_cost(f[0]['delay'])+get_layoff_cost(f[1]['delay'])
        
        for i in cabins:
            for j in cabins:
                if(f[0][i]<pnr['pax']+used_seat[f[0]['flight_id']][i] or f[1][j]<pnr['pax']+used_seat[f[0]['flight_id']][j]):
                    continue
                d1=cabin_ind[i]-cabin0
                d2=cabin_ind[j]-cabin0
                c=get_flight_time_score(d1,f[0]['flight_time'])
                c+=get_flight_time_score(d2,f[1]['flight_time'])
#                 print(c+cost)
                if(c+cost<val):
                    val=c+cost
                    f_id=[f[0]['flight_id'],f[1]['flight_id']]
                    c_id=[i,j]
                    
    for f in t_flights:
        
#         print(f[1]['delay'])
        cost=get_delay_cost(f[0]['delay'])+get_layoff_cost(f[1]['delay'])+get_layoff_cost(f[2]['delay'])
        
        for i in cabins:
            for j in cabins:
                for k in cabins:
                    if(f[0][i]<pnr['pax']+used_seat[f[0]['flight_id']][i] or f[1][j]<pnr['pax']+used_seat[f[1]['flight_id']][j]
                  or f[2][k]<pnr['pax']+used_seat[f[2]['flight_id']][k]):
                        continue
                    d1=cabin_ind[i]-cabin0
                    d2=cabin_ind[j]-cabin0
                    d3=cabin_ind[k]-cabin0
                    c=get_flight_time_score(d1,f[0]['flight_time'])
                    c+=get_flight_time_score(d2,f[1]['flight_time'])
                    c+=get_flight_time_score(d3,f[2]['flight_time'])
#                 print(c+cost)
                    if(c+cost<val):
                        val=c+cost
                        f_id=[f[0]['flight_id'],f[1]['flight_id'],f[2]['flight_id']]
                        c_id=[i,j,k]
     
    if val!=1e18:
        tot_cost+=val
    return f_id,c_id,val

In [121]:
# tot_cost=0
for i in sorted_pnr:
#     print(i)
    f_id=i['flight_id']
    flight_id,cabin_id,cost=obj(alt_flight[f_id],i)
#     break
#     print(flight_id)
#     print(cabin_id)
    if(len(flight_id)==3):
        allocated[i['pnr']]=[flight_id,cabin_id,cost]
        used_seat[flight_id[0]][cabin_id[0]]+=i['pax']
        used_seat[flight_id[1]][cabin_id[1]]+=i['pax']
        used_seat[flight_id[2]][cabin_id[2]]+=i['pax']
    elif(len(flight_id)==2):
        allocated[i['pnr']]=[flight_id,cabin_id,cost]
        used_seat[flight_id[0]][cabin_id[0]]+=i['pax']
        used_seat[flight_id[1]][cabin_id[1]]+=i['pax']
    elif(flight_id[0]!=-1):
        allocated[i['pnr']]=[flight_id[0],cabin_id[0],cost]
        used_seat[flight_id[0]][cabin_id[0]]+=i['pax']

1 day, 14:42:00
0:17:00
15:44:00
23:54:00
15:44:00
23:54:00
15:44:00
23:54:00
15:44:00
23:54:00
8:21:00
1 day, 0:50:00
21:32:00
23:54:00
21:32:00
23:54:00
21:32:00
23:54:00
21:32:00
23:54:00
1 day, 8:21:00
0:50:00
21:32:00
23:54:00
21:32:00
23:54:00
21:32:00
23:54:00
21:32:00
23:54:00
18:32:00
4:16:00
13:06:00
13:47:00
13:06:00
13:47:00
13:06:00
13:47:00
13:06:00
13:47:00
18:32:00
1 day, 4:16:00
13:06:00
13:47:00
13:06:00
13:47:00
13:06:00
13:47:00
13:06:00
13:47:00
1 day, 18:32:00
4:16:00
13:06:00
13:47:00
13:06:00
13:47:00
13:06:00
13:47:00
13:06:00
13:47:00
20:01:00
14:56:00
19:46:00
23:54:00
19:46:00
23:54:00
19:46:00
23:54:00
19:46:00
23:54:00
1 day, 14:42:00
0:55:00
5:32:00
15:44:00
1:43:00
20:28:00
15:44:00
1:43:00
20:28:00
15:44:00
1:43:00
20:28:00
15:44:00
1:43:00
20:28:00
15:44:00
1:43:00
20:28:00
15:44:00
1:43:00
20:28:00
15:44:00
1:43:00
20:28:00
15:44:00
1:43:00
20:28:00
8:21:00
7:09:00
22:26:00
21:32:00
3:08:00
20:28:00
21:32:00
3:08:00
20:28:00
21:32:00
3:08:00
20:28:00


In [122]:
#  allocated

In [123]:
pnr_flight={}
for i in sorted_pnr:
    pnr_flight[i['pnr']]=i['flight_id']

In [124]:
remaining_seat={}
for i in used_seat:
    k={}
    for j in used_seat[i]:
        k[j]=maximum_seat[i][j]-used_seat[i][j]
    remaining_seat[i]=k 
remaining_seat
sum=0
flight_cancel={}
for i in allocated: 
    if allocated[i]=='NULL':
        if pnr_flight[i] in flight_cancel:
            flight_cancel[pnr_flight[i]]+=1
        else:
            flight_cancel[pnr_flight[i]]=1 
#         for k in alt_flight[pnr_flight[i]]['n_flights']:
#             print(k)
#             print(remaining_seat[k['flight_id']])

In [125]:
flight_cancel

{'INV-ZZ-1594157': 50,
 'INV-ZZ-9711495': 13,
 'INV-ZZ-3193130': 20,
 'INV-ZZ-6129434': 29,
 'INV-ZZ-5481055': 12,
 'INV-ZZ-6235385': 25}

In [126]:
len(flight_cancel)

6

In [127]:
len(cancel['data'])

24

In [128]:
# maximum_seat

In [129]:
sum=0
for i in allocated: 
    if allocated[i]=='NULL' or allocated[i]==[]:
        sum+=1
len(allocated)

494

In [130]:
sum

149

In [131]:
x=len(allocated)-sum
x

345

In [132]:
tot_cost/x

3878314.4902661536

In [ ]:
tot_cost

1338018499.141823